<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Train_37.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notebook to Train and Evaluate ResNet and ViT on 100 and __% datasets residing in a single directory for Multi Class Classification

In [2]:
%cd "/content/Dataset(s)/joint_ferramenta/mulitmodal_img_enc_txt"

/content/Dataset(s)/joint_ferramenta/mulitmodal_img_enc_txt


# Load Data


In [ ]:
# import os

# # Define the path to your dataset folder
# dataset_folder = './images-train'

# # Define the path for the output text file
# output_file = './train_dataset.txt'

# def custom_sort_key(item):
#     return item.replace('@', '')

# with open(output_file, 'w') as f:
#     for label in sorted(os.listdir(dataset_folder), key=custom_sort_key):
#         label_path = os.path.join(dataset_folder, label)
#         if os.path.isdir(label_path):
#             class_name = label
#             image_files = sorted(os.listdir(label_path))
#             for image_file in image_files:
#                 image_path = os.path.join(label_path, image_file)
#                 f.write(f'{image_path}, {class_name}\n')

Train Dataset

In [3]:
import os

image_file_paths = []
image_genre_labels = []
enc_file_paths = []
enc_genre_labels = []

labels_file = "./train_dataset.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()

        if filename.endswith("_4.png"):
            enc_file_paths.append(filename)
            enc_genre_labels.append(labels)

        elif filename.endswith("_3.png"):
            image_file_paths.append(filename)
            image_genre_labels.append(labels)

In [4]:
len(image_file_paths), len(enc_file_paths)

(66141, 66141)

In [5]:
image_file_paths[0:5] , image_genre_labels[0:5]

(['./images-train/@accessori_saldatore/113267806.jpg_3.png',
  './images-train/@accessori_saldatore/115061965.jpg_3.png',
  './images-train/@accessori_saldatore/119306609.jpg_3.png',
  './images-train/@accessori_saldatore/119392789.jpg_3.png',
  './images-train/@accessori_saldatore/122140907.jpg_3.png'],
 ['@accessori_saldatore',
  '@accessori_saldatore',
  '@accessori_saldatore',
  '@accessori_saldatore',
  '@accessori_saldatore'])

In [6]:
enc_file_paths[0:5] , enc_genre_labels[0:5]

(['./images-train/@accessori_saldatore/113267806.jpg_4.png',
  './images-train/@accessori_saldatore/115061965.jpg_4.png',
  './images-train/@accessori_saldatore/119306609.jpg_4.png',
  './images-train/@accessori_saldatore/119392789.jpg_4.png',
  './images-train/@accessori_saldatore/122140907.jpg_4.png'],
 ['@accessori_saldatore',
  '@accessori_saldatore',
  '@accessori_saldatore',
  '@accessori_saldatore',
  '@accessori_saldatore'])

In [7]:
unique_labels = set(enc_genre_labels)
unique_labels_list = list(unique_labels)

print("Unique Labels:", unique_labels_list)

Unique Labels: ['@cacciavite_giravite_cercafase', '@ruota', '@maniglia_pomello', '@chiavistello', '@catena', '@spazzola', '@pistola', '@accessori_trapano', '@morsetto_strettoio', '@rubinetto_miscelatore_valvola', '@tassello', '@livella', '@nastro', '@pinzetta', '@brugola', '@accessori_saldatore', '@carrello', '@chiodo_rivetto', '@lame_sega_circolare', '@scaffale', '@banco', '@carta-vetrata_carta-abrasiva', '@vite_bullone', '@lama_seghetto_sega', '@lima_grattuggia_raspa', '@scala', '@pinza_punzonatrice_giratubo', '@armadio_cassettiera', '@seghetto_sega', '@cazzuola_frattone', '@segnaletica', '@torcia-faro', '@sega-a-tazza', '@collare', '@lucchetto', '@serratura_cilindro', '@metro_flessometro', '@bancale_cargo', '@colla_silicone', '@cerniera_bandella', '@chiave_inglese_poligonale', '@vernice', '@pennello', '@chiave-a-bussola_tubo', '@coltello', '@forbice_cesoia', '@estrattore', '@martello_mazzetta_ascia', '@cassaforte', '@scalpello', '@spatola_raschietto', '@staffa_squadretta']


In [8]:
num_classes = len(unique_labels_list)
num_classes

52

In [9]:
import random


percentage = 0.3

label_counts = {label: 0 for label in unique_labels_list}

num_samples_to_include_per_label = {}
for label in unique_labels_list:
    label_count = enc_genre_labels.count(label)
    num_samples_to_include = int(label_count * percentage)
    num_samples_to_include_per_label[label] = num_samples_to_include


filtered_enc_file_paths = []
filtered_enc_genre_labels = []


selected_samples_per_label = {label: 0 for label in unique_labels_list}
for path, label in zip(enc_file_paths, enc_genre_labels):
    if selected_samples_per_label[label] < num_samples_to_include_per_label[label]:
        filtered_enc_file_paths.append(path)
        filtered_enc_genre_labels.append(label)
        selected_samples_per_label[label] += 1

In [10]:
len(filtered_enc_file_paths)

19818

In [11]:
filtered_enc_file_paths[0:5]

['./images-train/@accessori_saldatore/113267806.jpg_4.png',
 './images-train/@accessori_saldatore/115061965.jpg_4.png',
 './images-train/@accessori_saldatore/119306609.jpg_4.png',
 './images-train/@accessori_saldatore/119392789.jpg_4.png',
 './images-train/@accessori_saldatore/122140907.jpg_4.png']

In [12]:
# Create new lists to combine data from both lists
train_file_paths = image_file_paths + filtered_enc_file_paths
train_genre_labels = image_genre_labels + filtered_enc_genre_labels

In [13]:
len(train_file_paths)

85959

Test Dataset

In [14]:
import os

test_image_file_paths = []
test_image_genre_labels = []
test_enc_file_paths = []
test_enc_genre_labels = []

test_labels_file = "./test_dataset.txt"

with open(test_labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()

        if filename.endswith("_4.png"):
            test_enc_file_paths.append(filename)
            test_enc_genre_labels.append(labels)

        elif filename.endswith("_3.png"):
            test_image_file_paths.append(filename)
            test_image_genre_labels.append(labels)

In [15]:
len(test_image_file_paths), len(test_enc_file_paths)

(21869, 21869)

In [16]:
test_image_file_paths[0:5] , test_image_genre_labels[0:5]

(['./images-val/@accessori_saldatore/123031564.jpg_3.png',
  './images-val/@accessori_saldatore/125066170.jpg_3.png',
  './images-val/@accessori_saldatore/128436452.jpg_3.png',
  './images-val/@accessori_saldatore/129979053.jpg_3.png',
  './images-val/@accessori_saldatore/130236413.jpg_3.png'],
 ['@accessori_saldatore',
  '@accessori_saldatore',
  '@accessori_saldatore',
  '@accessori_saldatore',
  '@accessori_saldatore'])

In [17]:
import random

label_counts = {label: 0 for label in unique_labels_list}

test_num_samples_to_include_per_label = {}
for label in unique_labels_list:
    label_count = test_enc_genre_labels.count(label)
    num_samples_to_include = int(label_count * percentage)
    test_num_samples_to_include_per_label[label] = num_samples_to_include


test_filtered_enc_file_paths = []
test_filtered_enc_genre_labels = []


test_selected_samples_per_label = {label: 0 for label in unique_labels_list}
for path, label in zip(test_enc_file_paths, test_enc_genre_labels):
    if test_selected_samples_per_label[label] < test_num_samples_to_include_per_label[label]:
        test_filtered_enc_file_paths.append(path)
        test_filtered_enc_genre_labels.append(label)
        test_selected_samples_per_label[label] += 1

In [18]:
len(test_filtered_enc_file_paths)

6533

In [19]:
# Create new lists to combine data from both lists
test_file_paths = test_image_file_paths + test_filtered_enc_file_paths
test_genre_labels = test_image_genre_labels + test_filtered_enc_genre_labels

In [20]:
len(test_file_paths)

28402

# ResNet

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path)
        label = int(self.labels[idx])
        # label = torch.tensor(label,dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = CustomDataset(train_file_paths, train_genre_labels, transform=transform)
test_dataset = CustomDataset(test_file_paths, test_genre_labels, transform=transform)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32  # Adjust as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
device = "cuda" if torch.cuda.is_available() else "cpu"

resnet = models.resnet101(pretrained=True)
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, num_classes)  # num_classes is the number of classes in your dataset
resnet.to(device)
print(resnet)


In [ ]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
num_epochs = 30

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
import numpy as np

In [ ]:
training_loss = []

for epoch in range(num_epochs):
    resnet.train()
    running_loss = 0.0


    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    training_loss.append(epoch_loss)

    print("Training Loss==========================>>")
    print(f"Epoch {epoch+1}/{num_epochs} Training Loss: {epoch_loss:.4f}")

    resnet.eval()
    predicted_classes = []
    actual_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = resnet(images)

            _, predicted_label = torch.max(outputs, 1)

            predicted_classes.extend(predicted_label.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())

    accuracy = accuracy_score(actual_labels, predicted_classes)
    # precision = precision_score(actual_labels, predicted_classes, average='weighted')
    # recall = recall_score(actual_labels, predicted_classes, average='weighted')
    # f1 = f1_score(actual_labels, predicted_classes, average='weighted')

    print("Accuracy:", accuracy)
    # print("Precision:", precision)
    # print("Recall:", recall)
    # print("F1-score:", f1)

    print(classification_report(actual_labels, predicted_classes))
    # print("Confusion Matrix: ")
    # cm = confusion_matrix(actual_labels, predicted_classes)
    # print(cm)
    predicted_classes = np.array(predicted_classes)
    actual_labels = np.array(actual_labels)
    print(roc_auc_score(actual_labels, predicted_classes))


    save_dir = "/content/Model/Models-Train-30/"
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

    model_name = str(epoch+1) + "_model.pth"
    save_path = os.path.join(save_dir, model_name)  # Specify the complete path to the model file
    torch.save(resnet.state_dict(), save_path)

# ViT

In [21]:
import torch
# !pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attentio

In [37]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/vit-base-patch16-224"
image_processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name).to(device)

loading configuration file preprocessor_config.json from cache at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3/preprocessor_config.json
size should be a dictionary on of the following set of keys: ({'width', 'height'}, {'shortest_edge'}, {'shortest_edge', 'longest_edge'}, {'longest_edge'}), got 224. Converted to {'height': 224, 'width': 224}.
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



Label Encoding

In [38]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
transformed_train_genre_labels = label_encoder.fit_transform(train_genre_labels)

In [ ]:
# label_to_id_mapping = {label: id for label, id in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
# label_to_id_mapping

In [39]:
label_encoder1 = LabelEncoder()
transformed_test_genre_labels = label_encoder1.fit_transform(test_genre_labels)

In [40]:
label_to_id_mapping = {label: id for label, id in zip(label_encoder1.classes_, label_encoder1.transform(label_encoder1.classes_))}
id_to_label_mapping = {id: label for id, label in enumerate(label_encoder1.classes_)}

In [41]:
# Convert int64 to int in id_to_label_mapping and label_to_id_mapping
id_to_label_mapping = {int(id): label for id, label in id_to_label_mapping.items()}
label_to_id_mapping = {label: int(id) for label, id in label_to_id_mapping.items()}

In [42]:
id_to_label_mapping

{0: '@accessori_saldatore',
 1: '@accessori_trapano',
 2: '@armadio_cassettiera',
 3: '@bancale_cargo',
 4: '@banco',
 5: '@brugola',
 6: '@cacciavite_giravite_cercafase',
 7: '@carrello',
 8: '@carta-vetrata_carta-abrasiva',
 9: '@cassaforte',
 10: '@catena',
 11: '@cazzuola_frattone',
 12: '@cerniera_bandella',
 13: '@chiave-a-bussola_tubo',
 14: '@chiave_inglese_poligonale',
 15: '@chiavistello',
 16: '@chiodo_rivetto',
 17: '@colla_silicone',
 18: '@collare',
 19: '@coltello',
 20: '@estrattore',
 21: '@forbice_cesoia',
 22: '@lama_seghetto_sega',
 23: '@lame_sega_circolare',
 24: '@lima_grattuggia_raspa',
 25: '@livella',
 26: '@lucchetto',
 27: '@maniglia_pomello',
 28: '@martello_mazzetta_ascia',
 29: '@metro_flessometro',
 30: '@morsetto_strettoio',
 31: '@nastro',
 32: '@pennello',
 33: '@pinza_punzonatrice_giratubo',
 34: '@pinzetta',
 35: '@pistola',
 36: '@rubinetto_miscelatore_valvola',
 37: '@ruota',
 38: '@scaffale',
 39: '@scala',
 40: '@scalpello',
 41: '@sega-a-tazza'

In [43]:
len(train_genre_labels), len(transformed_train_genre_labels)

(85959, 85959)

In [44]:
len(test_genre_labels), len(transformed_test_genre_labels)

(28402, 28402)

Dataset Creating

In [45]:
from datasets import Dataset

train_data = {'image': train_file_paths, 'label': transformed_train_genre_labels}
ds_train = Dataset.from_dict(train_data)

In [46]:
val_data = {'image': test_file_paths, 'label': transformed_test_genre_labels}
ds_val = Dataset.from_dict(val_data)

In [47]:
import PIL.Image as pil

def transform(examples):
  inputs = image_processor([pil.open(img).convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

In [48]:
train_dataset = ds_train.with_transform(transform)
val_dataset = ds_val.with_transform(transform)

In [49]:
train_dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 85959
})

In [50]:
val_dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 28402
})

In [51]:
for item in val_dataset:
  print(item['pixel_values'].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
0


In [52]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [ ]:
# load the ViT model
model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels_list),
    # id2label={str(i): c for i, c in enumerate(unique_labels_list)},
    id2label=id_to_label_mapping,
    label2id=label_to_id_mapping,
    # label2id={c: str(i) for i, c in enumerate(unique_labels_list)},
    ignore_mismatched_sizes=True,
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "@accessori_saldatore",
    "1": "@accessori_trapano",
    "2": "@armadio_cassettiera",
    "3": "@bancale_cargo",
    "4": "@banco",
    "5": "@brugola",
    "6": "@cacciavite_giravite_cercafase",
    "7": "@carrello",
    "8": "@carta-vetrata_carta-abrasiva",
    "9": "@cassaforte",
    "10": "@catena",
    "11": "@cazzuola_frattone",
    "12": "@cerniera_bandella",
    "13": "@chiave-a-bussola_tubo",
    "14": "@chiave_inglese_poligonale",
    "15": "@chiavistello",
    "16": "@chiodo_rivetto",
    "17": "@colla_

Evaluation Metric

In [ ]:
from evaluate import load
import numpy as np
from sklearn.metrics import roc_auc_score

# load the accuracy and f1 metrics from the evaluate module
accuracy = load("accuracy")
f1 = load("f1")

def compute_metrics(eval_pred):
  # compute the accuracy and f1 scores & return them
  accuracy_score = accuracy.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids)
  f1_score = f1.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids, average="macro")

  # auroc_score = roc_auc_score(eval_pred.label_ids, np.argmax(eval_pred.predictions, axis=1))
  # print(f"AUROC Score: {auroc_score:.4f}")

  return {**accuracy_score, **f1_score}

Training

In [ ]:
# !pip install accelerate -U
# !pip install transformers[torch]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/Model/Models-Train-37", # output directory
  per_device_train_batch_size=32, # batch size per device during training
  evaluation_strategy="steps",    # evaluation strategy to adopt during training
  num_train_epochs=20,             # total number of training epochs
  # fp16=True,                    # use mixed precision
  save_steps=8000,                # number of update steps before saving checkpoint
  eval_steps=8000,                # number of update steps before evaluating
  logging_steps=8000,             # number of update steps before logging
  # save_steps=50,
  # eval_steps=50,
  # logging_steps=50,
  save_total_limit=4,             # limit the total amount of checkpoints on disk
  remove_unused_columns=False,    # remove unused columns from the dataset
  push_to_hub=False,              # do not push the model to the hub
  report_to='tensorboard',        # report metrics to tensorboard
  load_best_model_at_end=True,    # load the best model at the end of training
)

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,                        # the instantiated 🤗 Transformers model to be trained
    args=training_args,                 # training arguments, defined above
    data_collator=collate_fn,           # the data collator that will be used for batching
    compute_metrics=compute_metrics,    # the metrics function that will be used for evaluation
    train_dataset=train_dataset,        # training dataset
    eval_dataset=val_dataset,           # evaluation dataset
    tokenizer=image_processor,          # the processor that will be used for preprocessing the images
)

In [42]:
trainer.train()

***** Running training *****
  Num examples = 85,959
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 53,740
  Number of trainable parameters = 85,838,644


Step,Training Loss,Validation Loss,Accuracy,F1
8000,0.415600,0.315445,0.918844,0.886528
16000,0.109500,0.340700,0.923914,0.892816


***** Running Evaluation *****
  Num examples = 28402
  Batch size = 8
Saving model checkpoint to /content/Model/Models-Train-37/checkpoint-8000
Configuration saved in /content/Model/Models-Train-37/checkpoint-8000/config.json
Model weights saved in /content/Model/Models-Train-37/checkpoint-8000/pytorch_model.bin
Image processor saved in /content/Model/Models-Train-37/checkpoint-8000/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 28402
  Batch size = 8
Saving model checkpoint to /content/Model/Models-Train-37/checkpoint-16000
Configuration saved in /content/Model/Models-Train-37/checkpoint-16000/config.json
Model weights saved in /content/Model/Models-Train-37/checkpoint-16000/pytorch_model.bin
Image processor saved in /content/Model/Models-Train-37/checkpoint-16000/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 28402
  Batch size = 8


Buffered data was truncated after reaching the output size limit.

# Best Model Selection

In [53]:
from torch.utils.data import DataLoader

batch_size = 32
valid_dataset_loader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [54]:
checkpoint = 8000

In [55]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

while checkpoint <= 32000:
    model = ViTForImageClassification.from_pretrained(f"/content/Model/Models-Train-37/checkpoint-{checkpoint}").to(device)
    image_processor = ViTImageProcessor.from_pretrained(f"/content/Model/Models-Train-37/checkpoint-{checkpoint}")
    model = model.to(device)
    model.eval()
    predictions, labels = [], []

    for batch in valid_dataset_loader:
        pixel_values = batch["pixel_values"].to(device)
        label_ids = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values, labels=label_ids)
        logits = outputs.logits.detach().cpu()

        predictions.extend(logits.argmax(dim=-1).tolist())
        labels.extend(label_ids.tolist())

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    print("Accuracy: ", accuracy)
    print("precision: ", precision)
    print("f1_score: ", f1)
    print("recall", recall)
    print(classification_report(labels, predictions))
    checkpoint = checkpoint + 8000

loading configuration file /content/Model/Models-Train-37/checkpoint-8000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "@accessori_saldatore",
    "1": "@accessori_trapano",
    "2": "@armadio_cassettiera",
    "3": "@bancale_cargo",
    "4": "@banco",
    "5": "@brugola",
    "6": "@cacciavite_giravite_cercafase",
    "7": "@carrello",
    "8": "@carta-vetrata_carta-abrasiva",
    "9": "@cassaforte",
    "10": "@catena",
    "11": "@cazzuola_frattone",
    "12": "@cerniera_bandella",
    "13": "@chiave-a-bussola_tubo",
    "14": "@chiave_inglese_poligonale",
    "15": "@chiavistello",
    "16": "@chiodo_rivetto",
    "17": "@colla_silicone",
    "18": "@collare",
    "19": "@coltello",
    "20": "@estrattore",
    "21": "@for

Accuracy:  0.9188437433983522
precision:  0.9174667305659775
f1_score:  0.9169665113383682
recall 0.9188437433983522
              precision    recall  f1-score   support

           0       0.69      0.47      0.56        76
           1       0.93      0.97      0.95      2442
           2       0.95      0.90      0.92       448
           3       0.97      0.98      0.98        62
           4       0.94      0.98      0.96       171
           5       0.90      0.81      0.85       228
           6       0.96      0.98      0.97      1389
           7       0.95      0.88      0.92       192
           8       0.97      0.96      0.96       861
           9       0.97      0.98      0.97       834
          10       0.85      0.93      0.89       176
          11       0.93      0.89      0.91        45
          12       0.83      0.83      0.83       145
          13       0.93      0.91      0.92      1094
          14       0.91      0.97      0.94      2021
          15      

All model checkpoint weights were used when initializing ViTForImageClassification.

All the weights of ViTForImageClassification were initialized from the model checkpoint at /content/Model/Models-Train-37/checkpoint-16000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ViTForImageClassification for predictions without further training.
loading configuration file /content/Model/Models-Train-37/checkpoint-16000/preprocessor_config.json
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

loading configuration file /content/Model/Models-Train-37/checkpoint-24000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base

Accuracy:  0.9239138088866982
precision:  0.9235248448328391
f1_score:  0.9233597870442207
recall 0.9239138088866982
              precision    recall  f1-score   support

           0       0.66      0.51      0.58        76
           1       0.96      0.95      0.96      2442
           2       0.96      0.92      0.94       448
           3       0.97      1.00      0.98        62
           4       0.91      0.98      0.94       171
           5       0.78      0.90      0.84       228
           6       0.97      0.97      0.97      1389
           7       0.95      0.90      0.93       192
           8       0.93      0.99      0.96       861
           9       0.95      0.99      0.97       834
          10       0.90      0.90      0.90       176
          11       0.87      0.87      0.87        45
          12       0.87      0.83      0.85       145
          13       0.90      0.94      0.92      1094
          14       0.95      0.96      0.95      2021
          15      

All model checkpoint weights were used when initializing ViTForImageClassification.

All the weights of ViTForImageClassification were initialized from the model checkpoint at /content/Model/Models-Train-37/checkpoint-24000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ViTForImageClassification for predictions without further training.
loading configuration file /content/Model/Models-Train-37/checkpoint-24000/preprocessor_config.json
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

loading configuration file /content/Model/Models-Train-37/checkpoint-32000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base

Accuracy:  0.9271882261812548
precision:  0.9273847517479491
f1_score:  0.9269816699128299
recall 0.9271882261812548
              precision    recall  f1-score   support

           0       0.55      0.61      0.58        76
           1       0.96      0.96      0.96      2442
           2       0.93      0.95      0.94       448
           3       0.98      1.00      0.99        62
           4       0.94      0.98      0.96       171
           5       0.91      0.83      0.87       228
           6       0.96      0.98      0.97      1389
           7       0.95      0.93      0.94       192
           8       0.96      0.97      0.97       861
           9       0.98      0.98      0.98       834
          10       0.85      0.90      0.88       176
          11       0.80      0.87      0.83        45
          12       0.85      0.83      0.84       145
          13       0.94      0.91      0.93      1094
          14       0.96      0.96      0.96      2021
          15      

All model checkpoint weights were used when initializing ViTForImageClassification.

All the weights of ViTForImageClassification were initialized from the model checkpoint at /content/Model/Models-Train-37/checkpoint-32000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ViTForImageClassification for predictions without further training.
loading configuration file /content/Model/Models-Train-37/checkpoint-32000/preprocessor_config.json
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



Accuracy:  0.9272234349693683
precision:  0.9276659166721101
f1_score:  0.9272198258381161
recall 0.9272234349693683
              precision    recall  f1-score   support

           0       0.61      0.51      0.56        76
           1       0.96      0.96      0.96      2442
           2       0.96      0.94      0.95       448
           3       0.98      1.00      0.99        62
           4       0.93      0.98      0.96       171
           5       0.90      0.85      0.87       228
           6       0.97      0.98      0.97      1389
           7       0.96      0.93      0.94       192
           8       0.98      0.96      0.97       861
           9       0.97      0.99      0.98       834
          10       0.90      0.89      0.89       176
          11       0.80      0.89      0.84        45
          12       0.90      0.83      0.86       145
          13       0.92      0.93      0.92      1094
          14       0.95      0.96      0.96      2021
          15      